# Train model 

# computed_features_historical_v3 -> training (80%)





In [ ]:
!pip install hopsworks==4.2.*

  Using cached hopsworks-4.2.9-py3-none-any.whl.metadata (11 kB)
  Using cached pyhumps-1.6.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached furl-2.1.4-py2.py3-none-any.whl.metadata (25 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 665.0/665.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 99.2 MB/s eta 0

In [ ]:
import hopsworks
from google.colab import userdata

HOPSWORKS_API_KEY = userdata.get('HOPSWORKS_API_KEY')

print(f'API key loaded')

In [ ]:
# ✅ Connect to your Hopsworks project
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()
print("✅ Connected to Hopsworks project successfully!")


In [ ]:
import os
import joblib
import pandas as pd
import numpy as np
import hopsworks
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# -------------------------------
# 1. CONNECT TO HOPSWORKS
# -------------------------------
project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()
print("✅ Connected to project:", project.name)

# -------------------------------
# 2. LOAD FEATURE GROUP
# -------------------------------
fg = fs.get_feature_group("computed_features_historical_v3", version=1)
df = fg.read()
print("✅ Loaded feature data. Shape:", df.shape)

# -------------------------------
# 3. PREPARE DATA
# -------------------------------
H = 72  # forecast horizon in hours
target_col = f"aqi_t_plus_{H}"

if target_col not in df.columns:
    raise ValueError(f"❌ Target column '{target_col}' not found in feature group.")

# Drop rows where target is missing
df_sup = df.dropna(subset=[target_col]).copy()

# Columns not to use as features
non_feature_cols = ["datetime", "timestamp"]
features = [c for c in df_sup.columns if c not in non_feature_cols + [target_col]]

print(f"📊 Total features before cleaning: {len(features)}")

# Handle missing / invalid input values safely
X = df_sup[features].copy()
X = X.replace([np.inf, -np.inf], np.nan)
X[X <= 0] = np.nan
X = X.ffill()

# Drop rows still having missing values
missing_before = X.isna().sum().sum()
X = X.dropna()
missing_after = X.isna().sum().sum()
print(f"🧹 Cleaned missing values: {missing_before} → {missing_after}")

# Align target with cleaned feature set
y = df_sup.loc[X.index, target_col].astype(float)

# -------------------------------
# 4. TRAINING DATA (80%) & LEAVE 20% FOR LATER TESTING
# -------------------------------
split_frac = 0.8
split_idx = int(len(X) * split_frac)

X_train, y_train = X.iloc[:split_idx], y.iloc[:split_idx]
X_test, y_test = X.iloc[split_idx:], y.iloc[split_idx:]  # reserved for later

print(f"✅ Training on first 80% of data: {len(X_train)} samples")
print(f"⏸ 20% reserved for later testing: {len(X_test)} samples")

# -------------------------------
# 5. TRAIN & EVALUATE MODELS
# -------------------------------
def metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

models = {
    "RandomForest": RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1),
    "Ridge": Ridge(alpha=1.0, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42)
}

results = []

for name, model in models.items():
    print(f"\n🚀 Training {name} ...")
    model.fit(X_train, y_train)
    # Evaluate on training set itself (optional) or validation subset
    preds = model.predict(X_train)
    mae, rmse, r2 = metrics(y_train, preds)
    results.append({"Model": name, "MAE": mae, "RMSE": rmse, "R2": r2})
    print(f"{name} -> MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.3f}")

# Compare models based on RMSE
results_df = pd.DataFrame(results).sort_values(by="RMSE")
print("\n📊 Model Comparison:\n", results_df)

best_model_name = results_df.iloc[0]["Model"]
best_model = models[best_model_name]
best_metrics = results_df.iloc[0]
print(f"\n🏆 Best Model: {best_model_name}")
print(f"📈 Metrics -> MAE: {best_metrics['MAE']:.2f}, RMSE: {best_metrics['RMSE']:.2f}, R²: {best_metrics['R2']:.3f}")




Connection closed.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1251499
✅ Connected to project: pearls_aqi_predictor
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.98s) 
✅ Loaded feature data. Shape: (6776, 155)
🔄 Converted scaled AQI columns to numeric AQI for 19 columns
📊 Total features before cleaning: 153
🧹 Cleaned missing values: 6 → 0
✅ Data prepared | Train: 5360, Val: 1341

🚀 Training RandomForest ...
RandomForest -> MAE: 19.64, RMSE: 36.11, R²: 0.748

🚀 Training Ridge ...
Ridge -> MAE: 39.87, RMSE: 55.88, R²: 0.397

🚀 Training GradientBoosting ...
GradientBoosting -> MAE: 30.83, RMSE: 46.31, R²: 0.586

📊 Model Comparison:
               Model        MAE       RMSE        R2
0      RandomForest  19.643446  36.113914  0.748284
2  GradientBoosting  30.832045  46.314794  0.585999
1             Ridge  39.867615  55.882982  0.397272

🏆 Best Model: RandomForest
📈 Metrics -> MAE: 19.64, RMSE: 36.11, R²: 0.748
💾 Model saved lo

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading /content/models/RandomForest_H72.pkl: 0.000%|          | 0/10913249 elapsed<00:00 remaining<?

Uploading /content/model_schema.json: 0.000%|          | 0/11837 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1251499/models/AQI_RandomForest_H72/1

✅ Registered model 'RandomForest' in Hopsworks (version 1)


In [ ]:
# -------------------------------
# 6. SAVE AND REGISTER BEST MODEL
# -------------------------------
# Save locally

model_dir = "models"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, "randomForest_model.pkl")
joblib.dump(best_model, model_path)
print(f"💾 Model saved locally at: {model_path}")

# Create schema
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

# Register model in Hopsworks
model = mr.sklearn.create_model(
    name="randomForest_test_3_model",
    metrics={
        "mae": float(best_metrics["MAE"]),
        "rmse": float(best_metrics["RMSE"]),
        "r2": float(best_metrics["R2"]),
    },
    model_schema=model_schema,
    description=f"{best_model_name} model trained for AQI prediction ({H}-hour horizon)"
)
model.save(model_path)
print(f"\n✅ Registered model 'randomForest_model' in Hopsworks (version {model.version})")

# computed_features_historical_v3 -> testing (20%)

In [ ]:
import hopsworks
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# --------------------
# ✅ Config / Helpers
# --------------------
USE_ZERO_FOR_NON_SELECTED = True
# If you prefer to replace non-selected features with the column mean instead of zero,
# set USE_ZERO_FOR_NON_SELECTED = False

def safe_metrics(y_true, y_pred):
    """Return (mae, rmse, r2). If not enough samples for r2, returns np.nan for r2."""
    if len(y_true) == 0:
        return (np.nan, np.nan, np.nan)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred) if len(y_true) >= 2 else np.nan
    return (mae, rmse, r2)

# --------------------
# ✅ Load Model
# --------------------

project = hopsworks.login()
mr = project.get_model_registry()

# Load your model (by name)
model = mr.get_model("randomForest_test_3_model", version=1)

model_dir = model.download()
model_path = model_dir + "/randomForest_model.pkl"

rf_model = joblib.load(model_path)
print("✅ Model loaded!")

# --------------------
# ✅ Load Feature Data
# --------------------
fs = project.get_feature_store()
fg = fs.get_feature_group("computed_features_historical_v3", version=1)
df = fg.read()

H = 72
target_col = f"aqi_t_plus_{H}"

df = df.dropna(subset=[target_col])
split_idx = int(len(df) * 0.8)
df_test = df.iloc[split_idx:].reset_index(drop=True)

non_feature_cols = ["datetime", "timestamp"]
features = [c for c in df_test.columns if c not in non_feature_cols + [target_col]]

X_full = df_test[features].copy()
y_full = df_test[target_col].copy()

print("✅ Test set:", X_full.shape)

# --------------------
# ✅ Feature Importance (from RF)
# --------------------
feature_importances = pd.Series(
    rf_model.feature_importances_,
    index=features
).sort_values(ascending=False)

print("\n🔥 Top 10 Features:\n", feature_importances.head(10))

# --------------------
# ✅ Generate feature counts: 5,10,15,... up to len(features)
# --------------------
step = 5
max_feats = len(features)
feature_counts = list(range(step, max_feats + 1, step))
if feature_counts[-1] != max_feats:
    feature_counts.append(max_feats)  # ensure the full set is included

print("\nWill evaluate feature counts:", feature_counts)

# --------------------
# ✅ Define AQI-level bins (low / normal / high) using tertiles of y_full
# This is data-driven (if you have fixed thresholds, replace with those).
# --------------------
q_low = np.nanpercentile(y_full, 33.3333)
q_high = np.nanpercentile(y_full, 66.6667)

def aqi_level_mask(y, level):
    """Return boolean mask for given level: 'low','normal','high' based on tertiles."""
    if level == "low":
        return y <= q_low
    elif level == "normal":
        return (y > q_low) & (y <= q_high)
    elif level == "high":
        return y > q_high
    else:
        raise ValueError("level must be 'low','normal' or 'high'")

print(f"\nAQI tertile cutpoints: low<= {q_low:.4f}, high> {q_high:.4f}")

# --------------------
# ✅ Iterate and evaluate
# --------------------
results = []
detailed_rows = []  # to store additional per-k details if needed

for k in feature_counts:
    top_k = feature_importances.head(k).index.tolist()

    # Create a copy with ALL features
    X_test = X_full.copy()

    # Set non-selected features to zero or column mean
    if USE_ZERO_FOR_NON_SELECTED:
        for col in X_test.columns:
            if col not in top_k:
                X_test[col] = 0.0
    else:
        # Replace with column means for non-selected features
        col_means = X_test.mean()
        for col in X_test.columns:
            if col not in top_k:
                X_test[col] = col_means[col]

    # Predict
    y_pred = rf_model.predict(X_test)

    # Overall metrics
    mae_all, rmse_all, r2_all = safe_metrics(y_full.values, y_pred)

    # Per-AQI-level metrics
    metrics_by_level = {}
    for lvl in ("low", "normal", "high"):
        mask = aqi_level_mask(y_full, lvl)
        y_true_lvl = y_full[mask].values
        y_pred_lvl = y_pred[mask]
        mae_lvl, rmse_lvl, r2_lvl = safe_metrics(y_true_lvl, y_pred_lvl)
        metrics_by_level[f"MAE_{lvl}"] = mae_lvl
        metrics_by_level[f"RMSE_{lvl}"] = rmse_lvl
        metrics_by_level[f"R2_{lvl}"] = r2_lvl

    results.append({
        "Features": k,
        "MAE": mae_all,
        "RMSE": rmse_all,
        "R2": r2_all,
        "MAE_low": metrics_by_level["MAE_low"],
        "RMSE_low": metrics_by_level["RMSE_low"],
        "R2_low": metrics_by_level["R2_low"],
        "MAE_normal": metrics_by_level["MAE_normal"],
        "RMSE_normal": metrics_by_level["RMSE_normal"],
        "R2_normal": metrics_by_level["R2_normal"],
        "MAE_high": metrics_by_level["MAE_high"],
        "RMSE_high": metrics_by_level["RMSE_high"],
        "R2_high": metrics_by_level["R2_high"],
    })

    print(f"✅ Top {k}: MAE={mae_all:.3f}, RMSE={rmse_all:.3f}, R²={r2_all:.4f} "
          f"(low RMSE={metrics_by_level['RMSE_low']:.3f}, normal RMSE={metrics_by_level['RMSE_normal']:.3f}, high RMSE={metrics_by_level['RMSE_high']:.3f})")

# --------------------
# ✅ Results DataFrame + Save
# --------------------
results_df = pd.DataFrame(results)
results_df = results_df.sort_values("Features").reset_index(drop=True)
print("\n📊 Results (first rows):\n", results_df.head())

# Save results for future analysis
results_df.to_csv("feature_count_evaluation_results.csv", index=False)
print("\nSaved results to feature_count_evaluation_results.csv")

# --------------------
# ✅ Plots
# --------------------
plt.figure(figsize=(9,5))
plt.plot(results_df["Features"], results_df["RMSE"], marker='o')
plt.title("Overall RMSE vs Number of Top Features")
plt.xlabel("Number of Top Features")
plt.ylabel("RMSE")
plt.grid()
plt.tight_layout()
plt.show()

# Class-wise RMSE plot (low, normal, high)
plt.figure(figsize=(9,5))
plt.plot(results_df["Features"], results_df["RMSE_low"], marker='o', label="Low AQI RMSE")
plt.plot(results_df["Features"], results_df["RMSE_normal"], marker='o', label="Normal AQI RMSE")
plt.plot(results_df["Features"], results_df["RMSE_high"], marker='o', label="High AQI RMSE")
plt.title("Class-wise RMSE vs Number of Top Features")
plt.xlabel("Number of Top Features")
plt.ylabel("RMSE")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

# --------------------
# ✅ Best feature counts
# --------------------
best_row_overall = results_df.loc[results_df["RMSE"].idxmin()]
print(f"\n🏆 Best Feature Count (overall RMSE): {int(best_row_overall['Features'])}")
print(best_row_overall)

# Best for each AQI class by RMSE
best_low = results_df.loc[results_df["RMSE_low"].idxmin()] if results_df["RMSE_low"].notnull().any() else None
best_normal = results_df.loc[results_df["RMSE_normal"].idxmin()] if results_df["RMSE_normal"].notnull().any() else None
best_high = results_df.loc[results_df["RMSE_high"].idxmin()] if results_df["RMSE_high"].notnull().any() else None

if best_low is not None:
    print(f"\n🏆 Best Feature Count for LOW AQI (RMSE): {int(best_low['Features'])}")
    print(best_low)
if best_normal is not None:
    print(f"\n🏆 Best Feature Count for NORMAL AQI (RMSE): {int(best_normal['Features'])}")
    print(best_normal)
if best_high is not None:
    print(f"\n🏆 Best Feature Count for HIGH AQI (RMSE): {int(best_high['Features'])}")
    print(best_high)

# --------------------
# ✅ Optional: show top features for the best overall k
# --------------------
best_k = int(best_row_overall["Features"])
print(f"\nTop {best_k} features:\n", feature_importances.head(best_k).index.tolist())
